In [139]:
from datetime import date
from prophet import Prophet
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
df = pd.read_csv("../Resources/btcjoin.csv", parse_dates=['date'])
df = df.drop(columns=['volume','change', 'low', 'high', 'open'])
df = df.rename(columns={"value": "wallets"})
df['price']=df['price'].str.replace(',','')
df['price']=df['price'].astype("float")
df.dtypes
df.to_csv('../Resources/colabbtc.csv')

#https://dcresearch.medium.com/metcalfes-law-and-bitcoin-s-value-2b99c7efd1fa

In [140]:
df['Metcafe']=df['address']**2
df['value'] = df['Metcafe']/df['mined']
df["value"] = df["value"].map("{:.2f}".format)
df['value']=df['value'].astype("float")
df['networkvalue'] = df["price"] - df["value"]
df.dtypes


date            datetime64[ns]
price                  float64
wallets                  int64
address                float64
mined                  float64
Metcafe                float64
value                  float64
networkvalue           float64
dtype: object

In [141]:
df

,date,price,wallets,address,mined,Metcafe,value,networkvalue
0,2010-09-25,0.1,2,656.0,4109450.00,4.303360e+05,0.10,0.00
1,2010-09-26,0.1,2,444.0,4110750.00,1.971360e+05,0.05,0.05
2,2010-09-27,0.1,2,1070.0,4132150.00,1.144900e+06,0.28,-0.18
3,2010-09-28,0.1,2,1503.0,4133200.00,2.259009e+06,0.55,-0.45
4,2010-09-29,0.1,2,1049.0,4148050.00,1.100401e+06,0.27,-0.17
...,...,...,...,...,...,...,...,...
4381,2022-10-21,19162.6,84879615,924681.0,19185681.25,8.550350e+11,44566.31,-25403.71
4382,2022-10-22,19204.8,84888147,830404.0,19186618.75,6.895708e+11,35940.19,-16735.39
4383,2022-10-23,19571.2,84893809,804140.0,19187600.00,6.466411e+11,33700.99,-14129.79
4384,2022-10-24,19331.5,84902680,919344.0,19188556.25,8.451934e+11,44046.74,-24715.24


In [142]:
df['status'] = df['networkvalue'].apply(lambda x: '1' if x > 0 else '0')
# Create our features
X = df.drop(columns="status")
X = pd.get_dummies(X)

# Create our target
X = df.drop(columns="date")
y = df['status']

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1, train_size=0.70)


In [143]:
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.metrics import balanced_accuracy_score
from sklearn.metrics import accuracy_score

In [144]:
## Logistic Regression
from sklearn.linear_model import LogisticRegression
model = LogisticRegression(solver='lbfgs', random_state=1)
LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
   intercept_scaling=1, max_iter=100, multi_class='warn', penalty='12',
   random_state=1, solver='lbfgs', tol=0.0001, warm_start=False)
model.fit(X_train, y_train)


LogisticRegression(random_state=1)

In [145]:
y_pred = model.predict(X_test)
# Calculating the confusion matrix
cm = confusion_matrix(y_test, y_pred)
# Create a DataFrame from the confusion matrix.
cm_df = pd.DataFrame(
    cm, index=["Actual 0", "Actual 1"], columns=["Predicted 0", "Predicted 1"])


In [146]:
print("Confusion Matrix")
display(cm_df)
print(f'Training Score: {model.score(X_train, y_train)}')
print(f'Testing Score: {model.score(X_test, y_test)}')
print(classification_report(y_test, y_pred))
print(balanced_accuracy_score(y_test, y_pred))
print(accuracy_score(y_test, y_pred))


Confusion Matrix


,Predicted 0,Predicted 1
Actual 0,1242,0
Actual 1,74,0


Training Score: 0.9368078175895765
Testing Score: 0.9437689969604863
              precision    recall  f1-score   support

           0       0.94      1.00      0.97      1242
           1       0.00      0.00      0.00        74

    accuracy                           0.94      1316
   macro avg       0.47      0.50      0.49      1316
weighted avg       0.89      0.94      0.92      1316

0.5
0.9437689969604863


c:\Users\16474\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1334: UndefinedMetricWarning:

Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.

c:\Users\16474\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1334: UndefinedMetricWarning:

Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.

c:\Users\16474\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1334: UndefinedMetricWarning:

Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.



In [147]:
## Over and Under sampling
# Resample the training data with SMOTEENN
from imblearn.combine import SMOTEENN
from imblearn.metrics import classification_report_imbalanced
smote_enn = SMOTEENN(random_state=1)
X_resampled, y_resampled = smote_enn.fit_resample(X, y)

# Train the Logistic Regression model using the resampled data
from sklearn.linear_model import LogisticRegression
# model = LogisticRegression(solver='lbfgs', random_state=1).fit(X_resampled, y_resampled)
model = LogisticRegression(solver='lbfgs', random_state=1)
LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
   intercept_scaling=1, max_iter=100, multi_class='warn', penalty='12',
   random_state=1, solver='lbfgs', tol=0.0001, warm_start=False)
model.fit(X_resampled, y_resampled)


LogisticRegression(random_state=1)

In [148]:
y_pred = model.predict(X_test)
# Calculating the confusion matrix
cm = confusion_matrix(y_test, y_pred)
# Create a DataFrame from the confusion matrix.
cm_df = pd.DataFrame(
    cm, index=["Actual 0", "Actual 1"], columns=["Predicted 0", "Predicted 1"])


In [149]:
# Displaying results
print(f'Training Score: {model.score(X_resampled, y_resampled)}')
print(f'Testing Score: {model.score(X_test, y_test)}')
print("Confusion Matrix")
display(cm_df)
print(balanced_accuracy_score(y_test, y_pred))
print(accuracy_score(y_test, y_pred))
print("Imbalanced Classification Report")
print(classification_report_imbalanced(y_test, y_pred))

Training Score: 0.6774907749077491
Testing Score: 0.8199088145896657
Confusion Matrix


,Predicted 0,Predicted 1
Actual 0,1037,205
Actual 1,32,42


0.7012556034295165
0.8199088145896657
Imbalanced Classification Report
                   pre       rec       spe        f1       geo       iba       sup

          0       0.97      0.83      0.57      0.90      0.69      0.49      1242
          1       0.17      0.57      0.83      0.26      0.69      0.46        74

avg / total       0.93      0.82      0.58      0.86      0.69      0.49      1316



In [150]:
import plotly.express as px
import plotly.graph_objects as go
fig = px.scatter(df, x="date", y="price", color="status", color_discrete_sequence=["grey", "orange"],
                 title="price")
				 

fig.update_yaxes(fixedrange=False)
fig.update_layout(title_text='Bitcoin Value Status')
fig.update_yaxes(type="log")
fig.update_xaxes(ticklabelposition="inside top", title="Date")
fig.update_yaxes(nticks=15)
fig.update_xaxes(nticks=50)
fig.update_layout(
    margin=dict(l=20, r=100, t=70, b=20),
)

fig.update_layout(legend=dict(
    yanchor="top",
    y=0.99,
    xanchor="right",
    x=0.5
))

fig.update_layout(
	updatemenus=[
		dict(
			active=0,
			buttons=list([
				dict(label="Both",
					method="update",
					args=[{"visible": [True, True, True]},
						{"title": "Both"}]),
				dict(label="Under Value",
					method="update",
					args=[{"visible": [True, False, False]},
						{"title": "Under Value",
							}]),
				
				dict(label="Over Value",
					method="update",
					args=[{"visible": [False, True, False]},
						{"title": "Over Value",
							}]),
			]),
		)
	])

fig.show()
fig.write_html('../static/status.html')

In [151]:

# # example of grid searching key hyperparametres for logistic regression
# from sklearn.datasets import make_blobs
# from sklearn.model_selection import RepeatedStratifiedKFold
# from sklearn.model_selection import GridSearchCV
# from sklearn.linear_model import LogisticRegression
# # define dataset
# # Create our target
# X = df.drop(columns="date")
# y = df['status']
# # define models and parameters
# model = LogisticRegression()
# solvers = ['newton-cg', 'lbfgs', 'liblinear']
# penalty = ['l2']
# c_values = [100, 10, 1.0, 0.1, 0.01]
# # define grid search
# grid = dict(solver=solvers,penalty=penalty,C=c_values)
# cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=3, random_state=1)
# grid_search = GridSearchCV(estimator=model, param_grid=grid, n_jobs=-1, cv=cv, scoring='accuracy',error_score=0)
# grid_result = grid_search.fit(X, y)
# # summarize results
# print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
# means = grid_result.cv_results_['mean_test_score']
# stds = grid_result.cv_results_['std_test_score']
# params = grid_result.cv_results_['params']
# for mean, stdev, param in zip(means, stds, params):
#     print("%f (%f) with: %r" % (mean, stdev, param))

In [152]:
## Logistic Regression
from sklearn.linear_model import LogisticRegression
from imblearn.metrics import classification_report_imbalanced
model = LogisticRegression(solver='newton-cg', random_state=1)
LogisticRegression(C=10.0, class_weight=None, dual=False, fit_intercept=True,
   intercept_scaling=1, max_iter=3000, multi_class='warn', penalty='12',
   random_state=1, solver='newton-cg', tol=0.0001, warm_start=False)
model.fit(X_train, y_train)


c:\Users\16474\anaconda3\lib\site-packages\scipy\optimize\linesearch.py:478: LineSearchWarning:

The line search algorithm did not converge

c:\Users\16474\anaconda3\lib\site-packages\scipy\optimize\linesearch.py:327: LineSearchWarning:

The line search algorithm did not converge

c:\Users\16474\anaconda3\lib\site-packages\sklearn\utils\optimize.py:210: ConvergenceWarning:

newton-cg failed to converge. Increase the number of iterations.



LogisticRegression(random_state=1, solver='newton-cg')

In [153]:
y_pred = model.predict(X_test)
# Calculating the confusion matrix
cm = confusion_matrix(y_test, y_pred)
# Create a DataFrame from the confusion matrix.
cm_df = pd.DataFrame(
    cm, index=["Actual 0", "Actual 1"], columns=["Predicted 0", "Predicted 1"])

In [154]:
print("Confusion Matrix")
display(cm_df)
print(f'Training Score: {model.score(X_train, y_train)}')
print(f'Testing Score: {model.score(X_test, y_test)}')
print(classification_report(y_test, y_pred))
print(balanced_accuracy_score(y_test, y_pred))
print(accuracy_score(y_test, y_pred))

Confusion Matrix


,Predicted 0,Predicted 1
Actual 0,1234,8
Actual 1,57,17


Training Score: 0.9527687296416938
Testing Score: 0.9506079027355623
              precision    recall  f1-score   support

           0       0.96      0.99      0.97      1242
           1       0.68      0.23      0.34        74

    accuracy                           0.95      1316
   macro avg       0.82      0.61      0.66      1316
weighted avg       0.94      0.95      0.94      1316

0.6116442529486008
0.9506079027355623


In [155]:
## Over and Under sampling
# Resample the training data with SMOTEENN
from imblearn.combine import SMOTEENN
from imblearn.metrics import classification_report_imbalanced
smote_enn = SMOTEENN(random_state=1)
X_resampled, y_resampled = smote_enn.fit_resample(X, y)

# Train the Logistic Regression model using the resampled data
from sklearn.linear_model import LogisticRegression
model = LogisticRegression(solver='newton-cg', random_state=1)
LogisticRegression(C=10.0, class_weight=None, dual=False, fit_intercept=True,
   intercept_scaling=1, max_iter=3000, multi_class='warn', penalty='12',
   random_state=1, solver='newton-cg', tol=0.0001, warm_start=False)
model.fit(X_resampled, y_resampled)

c:\Users\16474\anaconda3\lib\site-packages\scipy\optimize\linesearch.py:478: LineSearchWarning:

The line search algorithm did not converge

c:\Users\16474\anaconda3\lib\site-packages\scipy\optimize\linesearch.py:327: LineSearchWarning:

The line search algorithm did not converge

c:\Users\16474\anaconda3\lib\site-packages\sklearn\utils\optimize.py:210: ConvergenceWarning:

newton-cg failed to converge. Increase the number of iterations.



LogisticRegression(random_state=1, solver='newton-cg')

In [156]:
y_pred = model.predict(X_test)
# Calculating the confusion matrix
cm = confusion_matrix(y_test, y_pred)
# Create a DataFrame from the confusion matrix.
cm_df = pd.DataFrame(
    cm, index=["Actual 0", "Actual 1"], columns=["Predicted 0", "Predicted 1"])

In [157]:
# Displaying results
print(f'Training Score: {model.score(X_resampled, y_resampled)}')
print(f'Testing Score: {model.score(X_test, y_test)}')
print("Confusion Matrix")
display(cm_df)
print(balanced_accuracy_score(y_test, y_pred))
print(accuracy_score(y_test, y_pred))
print("Imbalanced Classification Report")
print(classification_report_imbalanced(y_test, y_pred))

Training Score: 0.7193726937269372
Testing Score: 0.8837386018237082
Confusion Matrix


,Predicted 0,Predicted 1
Actual 0,1118,124
Actual 1,29,45


0.7541345693519607
0.8837386018237082
Imbalanced Classification Report
                   pre       rec       spe        f1       geo       iba       sup

          0       0.97      0.90      0.61      0.94      0.74      0.56      1242
          1       0.27      0.61      0.90      0.37      0.74      0.53        74

avg / total       0.93      0.88      0.62      0.90      0.74      0.56      1316



In [163]:
with open('filename1.txt', 'w') as f:
        print(f'Training Score: {model.score(X_resampled, y_resampled)}',file=f)
        print(f'Testing Score: {model.score(X_test, y_test)}',file=f)
        print(("Confusion Matrix"),file=f)
        print(balanced_accuracy_score(y_test, y_pred),file=f)
        print(accuracy_score(y_test, y_pred),file=f)
        print(("Imbalanced Classification Report"),file=f)
        print(classification_report_imbalanced(y_test, y_pred), file=f)